In [31]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC3
from selenium.webdriver.common.action_chains import ActionChains
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
import requests
import time
from requests_html import HTMLSession

op = webdriver.ChromeOptions()
# op.add_argument('--headless=new')
prefs = {
    'profile.default_content_settings.popups': 0,
    'download.default_directory' : r"/home/administrator/cbs_bag_hold/data",
    'directory_upgrade': True
}
op.add_experimental_option('prefs' , prefs)
driver = webdriver.Chrome(options=op)


session = HTMLSession()


def login_flo():
    driver.get("http://10.24.1.71/mh-ops")
    time.sleep(5)
    username = driver.find_element(By.XPATH , "/html/body/div[2]/div[2]/div/div/form/div/div[4]/input[1]")
    username.send_keys("ca.2670054")

    password = driver.find_element(By.XPATH , "/html/body/div[2]/div[2]/div/div/form/div/div[4]/input[2]")
    password.send_keys("Chauhan@8091")
    time.sleep(2)
    try:
        cross = driver.find_element(By.XPATH , "/html/body/div[4]/div/button")
        cross.click()
    except:
        print("Cross Button Failed")
    time.sleep(1)
    submit = driver.find_element(By.XPATH , "/html/body/div[2]/div[2]/div/div/form/div/div[4]/div[4]/button/span")
    submit.click()
    time.sleep(10)

login_flo()
session_cookie = driver.get_cookies()
csrf_token = driver.execute_script("return document.querySelector('meta[name=csrf-token]').getAttribute('content');")

if csrf_token:
    session.headers.update({"csrf-token": csrf_token})
    print(csrf_token)
else:
    print("CSRF Token not found in page")
print(session_cookie)
selenium_user_agent = driver.execute_script("return navigator.userAgent;")
print(selenium_user_agent)
session.headers.update({"user-agent": selenium_user_agent})
for cookie in driver.get_cookies():
    session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain']) # Can't is set this here
   
payload = {'shipment_ids': ["OIPC0014533918", "shipment2"]} 
url = "http://10.24.1.71/mh-ops-routes-api/mh-ops-proxy/v1/track-and-trace-proxy/shipment/bulk-track/generate-request-id"
response = session.post(url , json=payload)

if response.status_code == 200:
    # Access and print the response content
    print(response.text)
else:
    # Print an error message if the request was not successful
    print(f"Request failed with status code {response.status_code}")


Cross Button Failed
wdqcsadD-9mAdbrHptnA2mcjiPKgy_gY0gPk
[{'domain': '10.24.1.71', 'expiry': 1742395623, 'httpOnly': False, 'name': '_ga_SV3K9FT76Q', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GS1.1.1707835620.1.1.1707835623.0.0.0'}, {'domain': '10.24.1.71', 'expiry': 1707922019, 'httpOnly': True, 'name': 'session', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'eyJtaC1vcHMiOnsiX19jdXJyZW50VVJMIjoiL21oLW9wcyIsInN0YXRlIjoiZWNiNDI5NmYtNWI4ZS00MjhiLTliNmUtYmE4YzAxMzk3ZWNkIiwibm9uY2UiOiIwMjU1NjYxMS00ZGZjLTRmZjQtOWRiMC1hOThhMjI0MDFmZTEiLCJ1c2VyIjp7InN1YiI6ImNhLjI2NzAwNTQiLCJuYW1lIjoiY2EuMjY3MDA1NCIsInVzZXJJZCI6ImNhLjI2NzAwNTQifSwiZXhwIjoxNzA3OTE4NDE5NzE5fSwiY3NyZlNlY3JldCI6IlQ2b0c2WG5yRjB2eWEtd2NHQllNU19PXyJ9'}, {'domain': '10.24.1.71', 'expiry': 1742395620, 'httpOnly': False, 'name': '_ga', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GA1.1.1352220609.1707835620'}, {'domain': '10.24.1.71', 'expiry': 1707922019, 'httpOnly': True, 'name': 'session.si